In [1]:
# Imports
import numpy as np
import pandas as pd
import os
import multiprocessing as mp
#from joblib import Parallel, delayed
import concurrent.futures
import pykov
import re
from fnmatch import fnmatch

# Method to match IP against flow srcIP
def sort_ip_flow(ip):
    
    # List to house flows when matches
    flows_list = []
    
    # Iterate over tcp_flows list
    for flow in flowdata_sample.iterrows():
        
        # Comparison logic - flow[1][3] corresponds to SrcIP in flow tuple
        if ip == flow[1][3]:
            
            # Append match to flows_list
            flows_list.append(flow)
    
    # Return dictionary of IPs and flows
    return {ip: flows_list}

In [2]:
# Import data

flowdata = pd.read_csv('/home/ehenry/CTU-13-Dataset/1/capture20110810.binetflow')

In [19]:
flowdata_sample = flowdata.sample(n=50000)

In [20]:
# create tuple of srcaddr, dstaddr, dstprt and proto
unique_src_addr = set()

for i in flowdata_sample.iterrows():
    unique_src_addr.add(i[1][3])

In [21]:
# Sort flows according to srcIP

ip_dicts = []

# Parallelization framework

with concurrent.futures.ProcessPoolExecutor() as executor:
    
    # pass in unique set to executor
    # Return dict from each process
    
    for d in executor.map(sort_ip_flow, unique_src_addr):
    
        # Roll all dicts up into list
        
        ip_dicts.append(d)

In [17]:
# List to house all flows with botnet label
botnets = []

# Iterate to find Botnet labeled flows and append to botnets list
for f in ip_dicts:
    for k,v in f.items():
        if (re.search('Botnet', v[0][1][14])):
            #temp = {}
            #temp[k] = v
            botnets.append(v)

In [22]:
botnets

[[(1508737, StartTime              2011/08/10 13:03:36.757754
   Dur                                             0
   Proto                                         udp
   SrcAddr                             147.32.84.165
   Sport                                        2079
   Dir                                            ->
   DstAddr                              89.108.104.3
   Dport                                          53
   State                                         INT
   sTos                                            0
   dTos                                          NaN
   TotPkts                                         1
   TotBytes                                       75
   SrcBytes                                       75
   Label        flow=From-Botnet-V42-UDP-Attempt-DNS
   Name: 1508737, dtype: object),
  (1674173, StartTime      2011/08/10 13:24:40.345105
   Dur                              0.085929
   Proto                                 udp
   SrcAddr        